In [ ]:
%load_ext autoreload
%autoreload 2

import os
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import seaborn as sns
import aequitas
from aequitas.group import Group
from aequitas.bias import Bias
from aequitas.fairness import Fairness
from aequitas.plotting import Plot
from aequitas.preprocessing import preprocess_input_df
import ast

import hivanalysis
import hivanalysis.utils as utils
import hivanalysis.get_models as get_models

import matplotlib.pyplot as plt
from sklearn.externals import joblib

import warnings
warnings.filterwarnings('ignore')

from IPython.display import Image
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

dburl = os.environ['DBURL']
engine = create_engine(dburl)
pd.set_option('display.max_columns',100)
path_to_models = '/gpfs/data/dsapp-lab/triage-production_runs_small/trained_models/'
path_to_matrices = '/gpfs/data/dsapp-lab/triage-production_runs_small/matrices/'

In [ ]:
pd.options.display.max_rows

-----

In [ ]:
best_access_12mo_mid = 94982
best_access_6mo_mid = None
best_access_12mo_mg_id = 21144

In [ ]:
def get_demographics():
    query = f"""with gender_race_info as (
        select distinct entity_id, 
            race_id, race,
            gender_id,
            case when transmission_category like '%%MSM%%' then 'MSM' else 'other' end as trans_categ_msm,
            transmission_category
        from features_cdph.demographics
        join features_cdph.trans_categ using (entity_id)
        join lookup_cdph.race using (race_id)
        )
        
        select model_id, p.entity_id, p.as_of_date,
                extract(year from p.as_of_date) as year,
                label_value, 
                race, gender_id as gender, 
                trans_categ_msm, transmission_category
        from test_results.predictions p
        join gender_race_info using (entity_id)
        join model_metadata.models using (model_id)
        where model_group_id = {best_access_12mo_mg_id}
        """
    demo = pd.read_sql(query, engine)
    return(demo)
demo = get_demographics() # just demographics; combine with predictions later
demo.head()

In [ ]:
def get_predictions(m_id):
    q = f"""
        select model_id, entity_id, as_of_date,
            score
        from test_results.predictions p
        where model_id ={m_id}
        """
    df = pd.read_sql(q, engine, parse_dates='as_of_date')
    return (df)

In [ ]:
def get_p_at_(m_id):
    q = f"""
        select value
        from test_results.evaluations e
        where metric='precision@' and parameter='10.0_pct'
        and model_id={m_id}
        """
    df = pd.read_sql(q, engine)
    return(df.loc[0].to_dict())

In [ ]:
def get_risk_categ(t):
    if 'MSM' in t and 'IDU' in t:
        return 'MSM_and_IDU'
    elif 'MSM' in t:
        return 'MSM'
    elif 'IDU' in t:
        return 'IDU'
    else:
        return 'Other'

In [ ]:
thresholds = {'rank_pct':[0.01]}

pred = get_predictions(best_access_12mo_mid)
#p_at_10 = get_p_at_(best_access_12mo_mid)
df = pd.merge(pred, demo, on=['entity_id', 'as_of_date'])
df = df.loc[df.groupby('entity_id')['label_value'].idxmax(), :]
df['risk'] = df['transmission_category'].apply(lambda x: get_risk_categ(x))
df = df[["entity_id", "score", "label_value", "race", "gender", "trans_categ_msm", "transmission_category", "risk"]]
g = Group()
xtab, _ = g.get_crosstabs(df, thresholds)

In [ ]:
xtab[['attribute_name', 'attribute_value', 'pp', 'group_size']]

In [ ]:
a = xtab[xtab.attribute_name=='race'][['attribute_name', 'attribute_value', 'pp', 'group_size']]
a['percent_pp'] = a.pp/a.pp.sum()*100
a['percent_group'] = a.group_size/a.group_size.sum()*100
a

In [ ]:
utils.get_modelhash_testmatrix(best_access_12mo_mg_id, 2046)
t =  pd.read_csv(path_to_matrices+'1eac69b12066fdbe4c81183a1818be92'+'.csv.gz', parse_dates=['as_of_date'])

In [ ]:
t.tail()

In [ ]:
pred['rank'] = pred['score'].rank(pct=True)
pred['label'] = pred['rank'] > 0.99

In [ ]:
p = pd.merge(pred, t, on =['entity_id', 'as_of_date'])

In [ ]:
p.groupby('label')['cd4_entity_id_6months_cd4_avg'].agg([np.mean, np.std])
p.groupby('label')['prevappts_entity_id_6months_completed_sum'].agg([np.mean, np.std])
p.groupby('label')['prevappts_entity_id_all_days_bn_appts_avg'].agg([np.mean, np.std])